In [1]:
# STEP 0: Setup & Spark
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "../../")))

from config import settings
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim

spark = SparkSession.builder \
    .appName("Synthetic_ETL_Pipeline") \
    .config("spark.jars", settings.JDBC_PATH) \
    .getOrCreate()

print("✅ Spark session started.")

25/06/22 23:51:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/Users/aryan/Desktop/project/venv/lib/python3.9/site-packages/google/rpc/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✅ Spark session started.


In [2]:
from config.spark_config import get_spark_session
from config import settings

# Step 3: Start Spark session
spark = get_spark_session(app_name="Read_Parquet_Tuning")

# Step 4: Define Parquet folder path
parquet_base = os.path.join(settings.BASE_DIR, "data", "parquet_data")

# Step 5: Read each file
parquet_files = {
    "title_basics": os.path.join(parquet_base, "title_basics.parquet"),
    "title_crew": os.path.join(parquet_base, "title_crew.parquet"),
    "title_episode": os.path.join(parquet_base, "title_episode.parquet"),
    "title_akas": os.path.join(parquet_base, "title_akas.parquet")
}

# Step 6: Show sample records
for name, path in parquet_files.items():
    print(f"📄 Reading {name}")
    df = spark.read.parquet(path)
    df.show(5)

25/06/22 23:51:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


📄 Reading title_basics
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------+---------+-------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|    genre_1|  genre_2|        ingested_at|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------+---------+-------------------+
|tt32444893|tvEpisode|McCraw vs. Herrin...|McCraw vs. Herrin...|      0|     2023|     \N|            \N| Reality-TV|     NULL|2025-06-22 23:11:21|
|tt32444895|tvEpisode|          Ed Genesis|          Ed Genesis|      0|     2019|     \N|            \N|Documentary|     NULL|2025-06-22 23:11:21|
|tt32444899|tvEpisode|    Aikens vs. Brown|    Aikens vs. Brown|      0|     2023|     \N|            \N| Reality-TV|     NULL|2025-06-22 23:11:21|
| tt3244490|tvEpisode|From Marshes to M...|From Marshes to M...|      0|     2002|     \N

In [3]:

import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "../../")))

from config import settings
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim
import os
from src.config.spark_config import get_spark_session
custom_tuning = {
    "spark.executor.memory": "6g",
    "spark.driver.memory": "4g",
    "spark.executor.cores": "4"
}
spark = get_spark_session(app_name="Read_Parquet_Tuning", custom_config=custom_tuning)




25/06/22 23:51:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
parquet_base = os.path.join(settings.BASE_DIR, "data", "parquet_data")

parquet_files = {
    "title_basics": os.path.join(parquet_base, "title_basics.parquet"),
    "title_crew": os.path.join(parquet_base, "title_crew.parquet"),
    "title_episode": os.path.join(parquet_base, "title_episode.parquet"),
    "title_akas": os.path.join(parquet_base, "title_akas.parquet")
}


In [5]:
from pyspark.sql.functions import col

for name, path in parquet_files.items():
    print(f"📄 Reading {name}")

    # Pushdown predicate
    df = spark.read \
        .option("spark.sql.parquet.filterPushdown", "true") \
        .parquet(path)

    # Filter as early as possible
    if "titleType" in df.columns:
        df = df.filter(col("titleType") == "movie")

    # Print schema and show top rows
    df.printSchema()
    df.show(5)

    # Example: count to test lazy eval and plan
    print("🔢 Record count:", df.count())

    # Performance plan
    df.explain(mode="formatted")

📄 Reading title_basics
root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genre_1: string (nullable = true)
 |-- genre_2: string (nullable = true)
 |-- ingested_at: string (nullable = true)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------+-------+-------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|    genre_1|genre_2|        ingested_at|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------+-------+-------------------+
|tt32444915|    movie|Lightning in a Bo...|Lightning in a Bo...|      0|       